# Unit Square
Here we show some example of plotting the objects defined on a Unit Square

In [ ]:
import dolfinx
from mpi4py import MPI
from fenicsx_plotly import plot

In [ ]:
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 3, 3)

In [ ]:
plot(mesh)

In [ ]:
plot(mesh, wireframe=False)

In [ ]:
#V = df.FunctionSpace(mesh, "CG",  3)
#p = df.Function(V)
#p.interpolate(df.Expression("sin(x[0])", degree=1))
#plot(p, scatter=True, wireframe=False)

In [ ]:
#plot(V)

In [ ]:
#plot(p, wireframe=False, show_grid=True)
#plot(p, wireframe=True, show_grid=True)

In [ ]:
#V = df.VectorFunctionSpace(mesh, "CG",  2)
#u = df.Function(V)
#u.interpolate(df.Expression(("1 + x[0]", "x[1]") , degree=1))
#plot(u, size=1)
#plot(u, normalize=True, size=0.1)
#plot(u, norm=True, size=0.6)

In [ ]:
#for component in ["magnitude", "x", "y"]:
#    plot(u, component=component)

In [ ]:
# Create a facet fuction in order to mark the subdomains
#ffun = df.MeshFunction("size_t", mesh, 2)
#ffun.set_all(0)

#fixed = df.CompiledSubDomain("x[0] < 0.5")
#free = df.CompiledSubDomain("x[0] >= 0.5")
# Mark the first subdomain with value 1

#fixed_marker = 1
#fixed.mark(ffun, fixed_marker)

# Mark the second subdomain with value 2
#free = df.CompiledSubDomain("near(x[0], 1) && on_boundary")
#free_marker = 2
#free.mark(ffun, free_marker)

#plot(ffun)

In [ ]:
#V = df.VectorFunctionSpace(mesh, "CG",  2)
#left = df.CompiledSubDomain("near(x[0], 0) && on_boundary")
#bc = df.DirichletBC(V, df.Constant((0.0, 0.0)), left)
#plot(bc, wireframe=True)